In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV


from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
df_full = pd.read_excel("..input/Data_Train.xlsx")
df_test = pd.read_excel("..input/Data_Test.xlsx")

#### Check the Training Dataset

In [3]:
df_full.head(10)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74
5,Hyundai EON LPG Era Plus Option,Hyderabad,2012,75000,LPG,Manual,First,21.1 km/kg,814 CC,55.2 bhp,5.0,2.35
6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,3.50
7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8.0,17.50
8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54 kmpl,1598 CC,103.6 bhp,5.0,5.20
9,Tata Indica Vista Quadrajet LS,Chennai,2012,65932,Diesel,Manual,Second,22.3 kmpl,1248 CC,74 bhp,5.0,1.95


In [4]:
df_full.shape

(6019, 12)

In [5]:
df_full.info()
df_full.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             5983 non-null   object 
 9   Power              5983 non-null   object 
 10  Seats              5977 non-null   float64
 11  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(8)
memory usage: 564.4+ KB


Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               2
Engine               36
Power                36
Seats                42
Price                 0
dtype: int64

#### Check the Testing Dataset

In [6]:
df_test.head(10)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti Alto K10 LXI CNG,Delhi,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0
1,Maruti Alto 800 2016-2019 LXI,Coimbatore,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0
2,Toyota Innova Crysta Touring Sport 2.4 MT,Mumbai,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0
3,Toyota Etios Liva GD,Hyderabad,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0
4,Hyundai i20 Magna,Mumbai,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0
5,Mahindra XUV500 W8 2WD,Coimbatore,2016,85609,Diesel,Manual,Second,16.0 kmpl,2179 CC,140 bhp,7.0
6,Toyota Fortuner 4x2 AT TRD Sportivo,Pune,2015,59000,Diesel,Automatic,First,12.55 kmpl,2982 CC,168.7 bhp,7.0
7,Hyundai EON Era Plus,Jaipur,2013,65000,Petrol,Manual,First,21.1 kmpl,814 CC,55.2 bhp,5.0
8,Honda City 1.5 S MT,Mumbai,2011,66000,Petrol,Manual,Second,17.0 kmpl,1497 CC,118 bhp,5.0
9,Mahindra XUV500 W6 2WD,Coimbatore,2015,54684,Diesel,Manual,First,15.1 kmpl,2179 CC,140 bhp,7.0


In [7]:
df_test.shape

(1234, 11)

In [8]:
df_test.info()
df_test.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               1234 non-null   object 
 1   Location           1234 non-null   object 
 2   Year               1234 non-null   int64  
 3   Kilometers_Driven  1234 non-null   int64  
 4   Fuel_Type          1234 non-null   object 
 5   Transmission       1234 non-null   object 
 6   Owner_Type         1234 non-null   object 
 7   Mileage            1234 non-null   object 
 8   Engine             1224 non-null   object 
 9   Power              1224 non-null   object 
 10  Seats              1223 non-null   float64
dtypes: float64(1), int64(2), object(8)
memory usage: 106.2+ KB


Name                  0
Location              0
Year                  0
Kilometers_Driven     0
Fuel_Type             0
Transmission          0
Owner_Type            0
Mileage               0
Engine               10
Power                10
Seats                11
dtype: int64

#### Split the Name column to get only the brand name

In [9]:
df_full['Name'] = df_full.Name.str.split().str.get(0)
df_test['Name'] = df_test.Name.str.split().str.get(0)

In [10]:
df_full.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


In [11]:
df_test.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats
0,Maruti,Delhi,2014,40929,CNG,Manual,First,32.26 km/kg,998 CC,58.2 bhp,4.0
1,Maruti,Coimbatore,2013,54493,Petrol,Manual,Second,24.7 kmpl,796 CC,47.3 bhp,5.0
2,Toyota,Mumbai,2017,34000,Diesel,Manual,First,13.68 kmpl,2393 CC,147.8 bhp,7.0
3,Toyota,Hyderabad,2012,139000,Diesel,Manual,First,23.59 kmpl,1364 CC,null bhp,5.0
4,Hyundai,Mumbai,2014,29000,Petrol,Manual,First,18.5 kmpl,1197 CC,82.85 bhp,5.0


#### Verifying

In [12]:
df_full['Name'].value_counts().sum()

6019

In [13]:
df_test['Name'].value_counts().sum()

1234

After comparing with the shape of the training set and the testing set, we can see that all names have show only the brand now

#### Finding the missing values

In [14]:
miss = [col for col in df_full.columns if df_full[col].isnull().any()]

print("Columns with missing values :")
print(miss)

Columns with missing values :
['Mileage', 'Engine', 'Power', 'Seats']


#### Fill Missing Values

In [15]:
df_full['Seats'].fillna(df_full['Seats'].mean(), inplace = True)
df_test['Seats'].fillna(df_full['Seats'].mean(), inplace = True)

data = pd.concat([df_full, df_test], sort = False)

In [16]:
df_full['Mileage'] = df_full['Mileage'].fillna('17.0 kmpl')
df_full['Mileage'] = df_full['Mileage'].replace("0.0 kmpl", "17.0 kmpl")

df_test['Mileage'] = df_test['Mileage'].fillna('17.0 kmpl')
df_test['Mileage'] = df_test['Mileage'].replace("0.0 kmpl", "17.0 kmpl")

df_full['Engine'] = df_full['Engine'].fillna('1197 CC')
df_test['Engine'] = df_full['Engine'].fillna('1197 CC')

df_full['Power'] = df_full['Power'].fillna('74 bhp')
df_test['Power'] = df_test['Power'].fillna('74 bhp')

df_full['Power'] = df_full['Power'].replace("null bhp", "74 bhp")
df_test['Power'] = df_test['Power'].replace("null bhp", "74 bhp")

In [17]:
df_full.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

In [18]:
df_full.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

Now that we have either filled the missing values or replaced 0 values with the average, we do not have any missing values

#### Extract float from object

In [19]:
def get_float(name):
    title_search = re.search('([\d+\.+\d]+\W)', name)
    
    if title_search:
        return title_search.group(1)
    
    return ""

In [20]:
df_full['Mileage'] = df_full['Mileage'].apply(get_float).astype('float')
df_full['Engine'] = df_full['Engine'].apply(get_float).astype('int')
df_full['Power'] = df_full['Power'].apply(get_float).astype('float')

df_test['Mileage'] = df_test['Mileage'].apply(get_float).astype('float')
df_test['Engine'] = df_test['Engine'].apply(get_float).astype('int')
df_test['Power'] = df_test['Power'].apply(get_float).astype('float')

df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6019 entries, 0 to 6018
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6019 non-null   float64
 8   Engine             6019 non-null   int32  
 9   Power              6019 non-null   float64
 10  Seats              6019 non-null   float64
 11  Price              6019 non-null   float64
dtypes: float64(4), int32(1), int64(2), object(5)
memory usage: 540.9+ KB


In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               1234 non-null   object 
 1   Location           1234 non-null   object 
 2   Year               1234 non-null   int64  
 3   Kilometers_Driven  1234 non-null   int64  
 4   Fuel_Type          1234 non-null   object 
 5   Transmission       1234 non-null   object 
 6   Owner_Type         1234 non-null   object 
 7   Mileage            1234 non-null   float64
 8   Engine             1234 non-null   int32  
 9   Power              1234 non-null   float64
 10  Seats              1234 non-null   float64
dtypes: float64(3), int32(1), int64(2), object(5)
memory usage: 101.4+ KB


#### Configuring the input

In [22]:
y = np.log1p(df_full.Price)    # log1p is used to make the predictions more accurate
X = df_full.drop(['Price'], axis = 1)    # Remove Price (since that is what we want to predict)

#### Split the dataset

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.80, test_size = 0.20, random_state = 1)

#### Label Encoding

In [24]:
label_encoder = LabelEncoder()

X_train['Name'] = label_encoder.fit_transform(X_train['Name'])
X_test['Name'] = label_encoder.fit_transform(X_test['Name'])
df_test['Name'] = label_encoder.fit_transform(df_test['Name'])

X_train['Location'] = label_encoder.fit_transform(X_train['Location'])
X_test['Location'] = label_encoder.transform(X_test['Location'])
df_test['Location'] = label_encoder.fit_transform(df_test['Location'])

X_train['Fuel_Type'] = label_encoder.fit_transform(X_train['Fuel_Type'])
X_test['Fuel_Type'] = label_encoder.transform(X_test['Fuel_Type'])
df_test['Fuel_Type'] = label_encoder.fit_transform(df_test['Fuel_Type'])

X_train['Transmission'] = label_encoder.fit_transform(X_train['Transmission'])
X_test['Transmission'] = label_encoder.transform(X_test['Transmission'])
df_test['Transmission'] = label_encoder.fit_transform(df_test['Transmission'])

X_train['Owner_Type'] = label_encoder.fit_transform(X_train['Owner_Type'])
X_test['Owner_Type'] = label_encoder.transform(X_test['Owner_Type'])
df_test['Owner_Type'] = label_encoder.fit_transform(df_test['Owner_Type'])

C:\Users\Rohan\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Rohan\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Rohan\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

X_train.head()

The data is successfully labelled

X_train.info()

---

# Machine Learning Models

---

## Scores

### 1. R<sup>2</sup> 

- In statistics, the coefficient of determination, denoted R<sup>2</sup> or r<sup>2</sup>, is the proportion of the variance in the dependent variable that is predictable from the independent variable(s).


### 2. Max_Error

- The margin of error is a statistic expressing the amount of random sampling error in the results of a survey. 
- The larger the margin of error, the less confidence one should have that a poll result would reflect the result of a survey of the entire population. 
- The margin of error will be positive whenever a population is incompletely sampled and the outcome measure has positive variance, which is to say, the measure varies. 

---

## 1. Gradient Boosting Regression

In [27]:
print(__doc__)

gbr1 = GradientBoostingRegressor()

# Set the parameters by cross-validation
parameters = {'loss': ['ls', 'quantile'],    # loss function to be optimized
              'learning_rate': [0.01, 0.1, 0.5, 1],    # Set how fast the model converges
              'n_estimators': [100, 500, 1000, 5000],    # The number of boosting stages to perform
              'max_depth': [2, 3, 5, 10]}    # The maximum depth of the individual regression estimators

scores = ['r2', 'max_error']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    gbr = GridSearchCV(gbr1,
                       parameters,
                       cv = 5,
                       n_jobs = 5,
                       verbose = True,
                       scoring = score)

    gbr.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(gbr.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = gbr.cv_results_['mean_test_score']
    stds = gbr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gbr.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, gbr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for r2

Fitting 5 folds for each of 128 candidates, totalling 640 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.2min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed: 12.1min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 30.5min
[Parallel(n_jobs=5)]: Done 640 out of 640 | elapsed: 43.8min finished


Best parameters set found on development set:

{'learning_rate': 0.1, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 5000}


Grid scores on development set:

0.629 (+/-0.014) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 100}
0.878 (+/-0.024) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 500}
0.900 (+/-0.025) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 1000}
0.935 (+/-0.019) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 5000}
0.697 (+/-0.017) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 100}
0.901 (+/-0.026) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 500}
0.921 (+/-0.025) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 1000}
0.948 (+/-0.016) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 5000}
0.751 (+/-0.014) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth':

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.5min
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed: 13.9min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed: 32.1min
[Parallel(n_jobs=5)]: Done 640 out of 640 | elapsed: 45.9min finished


Best parameters set found on development set:

{'learning_rate': 0.5, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 100}


Grid scores on development set:

-2.060 (+/-0.434) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 100}
-2.147 (+/-0.902) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 500}
-2.119 (+/-1.165) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 1000}
-1.916 (+/-1.265) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 2, 'n_estimators': 5000}
-2.037 (+/-0.365) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 100}
-2.079 (+/-1.290) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 500}
-1.961 (+/-1.445) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 1000}
-1.778 (+/-1.258) for {'learning_rate': 0.01, 'loss': 'ls', 'max_depth': 3, 'n_estimators': 5000}
-2.005 (+/-0.419) for {'learning_rate': 0.01, 'loss': 'ls', 'max

- Gradient boosting is a machine learning technique for regression and classification problems, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees.
- t builds the model in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function.

- Gradient Boosting involves three elements :-
1. A Loss function to be optimized
2. A weak learner to make predictions
3. An additive model to add weak learners to minimize the loss function

---

## 2. XGB Regression

In [28]:
print(__doc__)

xgb1 = XGBRegressor()

# Set the parameters by cross-validation
parameters = {'nthread' : [5],    # Uses threading to pun processes in parallel
             'objective' : ['reg:linear'],
             'learning_rate' : [0.01, 0.1, 0.5, 1],    # Set how fast the model converges
             'max_depth' : [15, 20, 25],     # Set the maximum depth of the individual regression estimators
             'min_child_weight' : [0.1, 0.5, 1, 1.5],    # Minimum sum of instance weight (hessian) needed in a child
             'silent' : [1],    # Whether to display output or not
             'subsample' : [0.7],    # Subsample ratio of the training instances
             'colsample_bytree' : [0.7],    # Used for subsampling of columns
             'n_estimators' : [10, 20, 50, 100]}    # The number of boosting stages to perform

scores = ['r2', 'max_error']

for score in scores:
    print("Tuning Hyperparameters for %s" % score)
    print()
    
    xgb = GridSearchCV(xgb1,
                      parameters,
                      cv = 7,
                      n_jobs = 7,
                      verbose = True,
                      scoring = score)
    
    xgb.fit(X_train, y_train)
    
    print("Best parameters set found on development set : ")
    print()
    print(xgb.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = xgb.cv_results_['mean_test_score']
    stds = xgb.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, xgb.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, xgb.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

Automatically created module for IPython interactive environment
Tuning Hyperparameters for r2

Fitting 7 folds for each of 192 candidates, totalling 1344 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    5.7s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:   28.5s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:  1.2min
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:  2.7min
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed:  4.6min
[Parallel(n_jobs=7)]: Done 1344 out of 1344 | elapsed:  5.1min finished


Best parameters set found on development set : 

{'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 25, 'min_child_weight': 1.5, 'n_estimators': 100, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


Grid scores on development set:

-3.236 (+/-0.360) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_estimators': 10, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-2.507 (+/-0.302) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_estimators': 20, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-1.003 (+/-0.182) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_estimators': 50, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
0.181 (+/-0.090) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_est

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:    4.9s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:   28.4s
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:  1.2min
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:  3.0min
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed:  5.2min
[Parallel(n_jobs=7)]: Done 1344 out of 1344 | elapsed:  5.8min finished


Best parameters set found on development set : 

{'colsample_bytree': 0.7, 'learning_rate': 0.1, 'max_depth': 25, 'min_child_weight': 1.5, 'n_estimators': 100, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


Grid scores on development set:

-3.760 (+/-0.611) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_estimators': 10, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-3.496 (+/-0.599) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_estimators': 20, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-2.839 (+/-0.591) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_estimators': 50, 'nthread': 5, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}
-2.205 (+/-0.623) for {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 15, 'min_child_weight': 0.1, 'n_es

- It is an abbreviation of Extreme Gradient Boosting
- It is a decision-tree based ensemble Machine Learning Algorithm that uses a gradient boosting framework
- When it comes to small-to-medium structured/tabular data, decision tree based algorithms are considered best-in-class right now
- Think of XGBoost as gradient boosting on ‘steroids’ (well it is called ‘Extreme Gradient Boosting’ for a reason!). It is a perfect combination of software and hardware optimization techniques to yield superior results using less computing resources in the shortest amount of time.

##### Advantages
1. Parallalization
2. Tree Pruning (Depth-first approach)
3. Hardware Optimization
4. Regularization (Prevent Overfitting)
5. Cross-Validation (Built in CV)

---

## 3. Decision Tree Regression

In [29]:
print(__doc__)


# Set the parameters by cross-validation
parameters={'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2)}

dtR= DecisionTreeRegressor()

scores = ['r2', 'max_error']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    gbr = GridSearchCV(dtR,
                       parameters,
                       cv = 5,
                       n_jobs = 5,
                       verbose = True,
                       scoring = score)

    gbr.fit( X_train, y_train)


    print("Best parameters set found on development set:")
    print()
    print(gbr.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = gbr.cv_results_['mean_test_score']
    stds = gbr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, gbr.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, gbr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print("Root Mean Squared Error(R^2) is :", np.sqrt(mse))
    print()
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for r2

Fitting 5 folds for each of 250 candidates, totalling 1250 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  80 tasks      | elapsed:    0.4s
[Parallel(n_jobs=5)]: Done 1250 out of 1250 | elapsed:    4.9s finished
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


Best parameters set found on development set:

{'max_depth': 11, 'min_samples_split': 10}


Grid scores on development set:

0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 10}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 30}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 50}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 70}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 90}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 110}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 130}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 150}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 170}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 190}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 210}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 230}
0.484 (+/-0.034) for {'max_depth': 1, 'min_samples_split': 250}
0.484 (+/-0.034) for {'max_depth': 1, 'min_sampl

[Parallel(n_jobs=5)]: Done 150 tasks      | elapsed:    0.4s


Best parameters set found on development set:

{'max_depth': 1, 'min_samples_split': 10}


Grid scores on development set:

-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 10}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 30}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 50}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 70}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 90}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 110}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 130}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 150}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 170}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 190}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 210}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 230}
-2.021 (+/-0.452) for {'max_depth': 1, 'min_samples_split': 250}
-2.021 (+/-0.452) for {'max_depth': 

[Parallel(n_jobs=5)]: Done 1250 out of 1250 | elapsed:    4.9s finished


The Decision tree is the simplest, yet the most powerful algorithm in machine learning.
 
 Decision tree regression is used for the continuous output problem. Continuous output means the output of the result is not discrete, i.e., it is not represented just by a discrete, known set of numbers or values. Decision tree regression observes features of an object and trains a model in the structure of a tree to predict data in the future to produce meaningful continuous output.

 Advantages:-

*   A small change in the data can cause a large change in the structure of the decision tree causing instability.
*   For a Decision tree sometimes calculation can go far more complex compared to other algorithms.
*   Decision tree often involves higher time to train the model.
*   Decision tree training is relatively expensive as complexity and time taken is more.
*   Decision Tree algorithm is inadequate for applying regression and predicting continuous values 

---

## 4. AdaBoost Regression

In [30]:
print(__doc__)

parameters =  {'base_estimator__max_depth': range(1, 5), 'n_estimators' : range(10, 200, 50), 'learning_rate' : [1] }

Ada= AdaBoostRegressor(base_estimator=DecisionTreeRegressor(), # Making object of AdaBoost regressor based on Decision tree regressor to get
                                                               #more precise results
                              random_state=0)
scores = ['r2', 'max_error']

for score in scores:
    print("Tuning Hyperparameters for %s" % score)
    print()
        
    Agbr = GridSearchCV(Ada,
                       parameters,
                       cv = 5,
                       n_jobs = 5,
                       verbose = True,
                       scoring = score)

    Agbr.fit( X_train, y_train)

    print("Best parameters set found on development set : ")
    print()
    print(Agbr.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = Agbr.cv_results_['mean_test_score']
    stds =  Agbr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, Agbr.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, Agbr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print("Root Mean Squared Error(R^2) is :", np.sqrt(mse))
    print()
    print()
    

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


Automatically created module for IPython interactive environment
Tuning Hyperparameters for r2

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    3.2s
[Parallel(n_jobs=5)]: Done  80 out of  80 | elapsed:   10.3s finished


Best parameters set found on development set : 

{'base_estimator__max_depth': 4, 'learning_rate': 1, 'n_estimators': 160}


Grid scores on development set:

0.550 (+/-0.038) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 10}
0.573 (+/-0.038) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 60}
0.573 (+/-0.038) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 110}
0.573 (+/-0.038) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 160}
0.722 (+/-0.037) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 10}
0.642 (+/-0.139) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 60}
0.606 (+/-0.173) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 110}
0.591 (+/-0.188) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 160}
0.784 (+/-0.031) for {'base_estimator__max_depth': 3, 'learning_rate': 1, 'n_estimator

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  70 tasks      | elapsed:    7.5s
[Parallel(n_jobs=5)]: Done  80 out of  80 | elapsed:   10.3s finished


Best parameters set found on development set : 

{'base_estimator__max_depth': 3, 'learning_rate': 1, 'n_estimators': 110}


Grid scores on development set:

-1.920 (+/-0.421) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 10}
-1.886 (+/-0.400) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 60}
-1.886 (+/-0.400) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 110}
-1.886 (+/-0.400) for {'base_estimator__max_depth': 1, 'learning_rate': 1, 'n_estimators': 160}
-1.944 (+/-0.594) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 10}
-1.989 (+/-0.853) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 60}
-1.938 (+/-0.952) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 110}
-1.935 (+/-0.943) for {'base_estimator__max_depth': 2, 'learning_rate': 1, 'n_estimators': 160}
-2.043 (+/-1.094) for {'base_estimator__max_depth': 3, 'learning_rate': 1, 'n_

An AdaBoost regressor is a meta-estimator that begins by fitting a regressor on the original dataset and then fits additional copies of the regressor on the same dataset but where the weights of instances are adjusted according to the error of the current prediction. As such, subsequent regressors focus more on difficult cases.

Advantages:-


*   Very good use of weak classifiers for cascading;
*   Different classification algorithms can be used as weak classifiers;
*   AdaBoost has a high degree of precision;
*   Relative to the bagging algorithm andRandom ForestAlgorithm, AdaBoost fully considers the weight of each classifier



---

## 5. Support Vector Regression

In [31]:
print(__doc__)


# Set the parameters by cross-validation
parameters = {'kernel': ['rbf'], 
              'gamma': [0.001, 0.0001],
              'C': [1, 10, 100, 1000], 
              'max_iter': [-1, 1]}

scores = [ 'max_error','r2']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    svr = GridSearchCV(SVR(), 
                       parameters,
                       cv = 5,
                       n_jobs = 5,
                       verbose = True,
                       scoring = score)

    svr.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(svr.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = svr.cv_results_['mean_test_score']
    stds = svr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, svr.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, svr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for max_error

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   14.5s
[Parallel(n_jobs=5)]: Done  80 out of  80 | elapsed:   31.7s finished
C:\Users\Rohan\anaconda3\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.


Best parameters set found on development set:

{'C': 1, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': 1}


Grid scores on development set:

-2.620 (+/-0.484) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1}
-2.310 (+/-0.290) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': 1}
-2.421 (+/-0.410) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': -1}
-2.310 (+/-0.290) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': 1}
-2.574 (+/-0.474) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1}
-2.375 (+/-0.131) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': 1}
-2.386 (+/-0.414) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': -1}
-2.375 (+/-0.131) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': 1}
-2.576 (+/-0.474) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1}
-2.375 (+/-0.131) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': 1}
-2.419 (+/-0.523) for {'C': 100, 'gamma': 0.0001, 'ker

[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   14.8s
[Parallel(n_jobs=5)]: Done  80 out of  80 | elapsed:   29.8s finished


Best parameters set found on development set:

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': -1}


Grid scores on development set:

0.187 (+/-0.033) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1}
-0.536 (+/-0.246) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': 1}
0.309 (+/-0.042) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': -1}
-0.535 (+/-0.247) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': 1}
0.197 (+/-0.027) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1}
-0.867 (+/-0.268) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': 1}
0.310 (+/-0.041) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': -1}
-0.866 (+/-0.266) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf', 'max_iter': 1}
0.193 (+/-0.025) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': -1}
-0.867 (+/-0.268) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf', 'max_iter': 1}
0.299 (+/-0.052) for {'C': 100, 'gamma': 0.0001, 'kernel

- Support Vector Machine are supervised learning models with associated learning algorithms that analyze data used for classification and regression analysis.
-In addition to performing linear classification, SVMs can efficiently perform a non-linear classification using what is called the kernel trick, implicitly mapping their inputs into high-dimensional feature spaces.
- it applies the statistics of support vectors, developed in the support vector machines algorithm, to categorize unlabeled data, and is one of the most widely used clustering algorithms in industrial applications.


---

## 6. Random Forest Regression

In [32]:
print(__doc__)


# Set the parameters by cross-validation
parameters = [{'n_estimators': [10, 20, 50, 100],'max_depth': [None, 100, 1000, 10000], 'max_features': [None, 10, 15, 25]}]

scores = [ 'max_error','r2']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    rfr = GridSearchCV(RandomForestRegressor(), 
                       parameters,
                       cv = 5,
                       n_jobs = 5,
                       verbose = True,
                       scoring = score)

    rfr.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(rfr.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = rfr.cv_results_['mean_test_score']
    stds = rfr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, rfr.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, rfr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

Automatically created module for IPython interactive environment
# Tuning hyper-parameters for max_error

Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   10.4s
[Parallel(n_jobs=5)]: Done 230 tasks      | elapsed:   29.5s
[Parallel(n_jobs=5)]: Done 320 out of 320 | elapsed:   39.7s finished


Best parameters set found on development set:

{'max_depth': 10000, 'max_features': None, 'n_estimators': 20}


Grid scores on development set:

-2.173 (+/-1.330) for {'max_depth': None, 'max_features': None, 'n_estimators': 10}
-2.226 (+/-1.247) for {'max_depth': None, 'max_features': None, 'n_estimators': 20}
-2.099 (+/-1.389) for {'max_depth': None, 'max_features': None, 'n_estimators': 50}
-2.089 (+/-1.315) for {'max_depth': None, 'max_features': None, 'n_estimators': 100}
-2.030 (+/-1.394) for {'max_depth': None, 'max_features': 10, 'n_estimators': 10}
-2.195 (+/-1.175) for {'max_depth': None, 'max_features': 10, 'n_estimators': 20}
-2.080 (+/-1.390) for {'max_depth': None, 'max_features': 10, 'n_estimators': 50}
-2.083 (+/-1.339) for {'max_depth': None, 'max_features': 10, 'n_estimators': 100}
nan (+/-nan) for {'max_depth': None, 'max_features': 15, 'n_estimators': 10}
nan (+/-nan) for {'max_depth': None, 'max_features': 15, 'n_estimators': 20}
nan (+/-nan) for {'max_depth': None

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:    8.7s
[Parallel(n_jobs=5)]: Done 230 tasks      | elapsed:   30.3s
[Parallel(n_jobs=5)]: Done 320 out of 320 | elapsed:   42.0s finished


Best parameters set found on development set:

{'max_depth': 1000, 'max_features': 10, 'n_estimators': 100}


Grid scores on development set:

0.914 (+/-0.020) for {'max_depth': None, 'max_features': None, 'n_estimators': 10}
0.919 (+/-0.026) for {'max_depth': None, 'max_features': None, 'n_estimators': 20}
0.924 (+/-0.024) for {'max_depth': None, 'max_features': None, 'n_estimators': 50}
0.923 (+/-0.022) for {'max_depth': None, 'max_features': None, 'n_estimators': 100}
0.918 (+/-0.023) for {'max_depth': None, 'max_features': 10, 'n_estimators': 10}
0.922 (+/-0.027) for {'max_depth': None, 'max_features': 10, 'n_estimators': 20}
0.926 (+/-0.025) for {'max_depth': None, 'max_features': 10, 'n_estimators': 50}
0.926 (+/-0.024) for {'max_depth': None, 'max_features': 10, 'n_estimators': 100}
nan (+/-nan) for {'max_depth': None, 'max_features': 15, 'n_estimators': 10}
nan (+/-nan) for {'max_depth': None, 'max_features': 15, 'n_estimators': 20}
nan (+/-nan) for {'max_depth': None, 'max_fea

- Random Decision Forests are an ensemble learning method for classification, regression and other tasks that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.

- Random forests (RF) construct many individual decision trees at training. Predictions from all trees are pooled to make the final prediction; the mode of the classes for classification or the mean prediction for regression.

- This model uses two key concepts that gives it the name random:
1. Random sampling of training data points when building trees
2. Random subsets of features considered when splitting nodes

---

## 7. Bagging Regressor

In [34]:
print(__doc__)

xgb1 = BaggingRegressor()

# Set the parameters by cross-validation
parameters = [{'n_estimators': [5, 10, 50, 100], 'max_samples': [1, 5, 10, 20],
                     'max_features': [10], 'random_state': [None, 4, 8]}]


scores = ['r2', 'max_error']

for score in scores:
    print("Tuning Hyperparameters for %s" % score)
    print()
    
    bag= GridSearchCV(xgb1,
                      parameters,
                      cv = 7,
                      n_jobs = 7,
                      verbose = True,
                      scoring = score)
    
    bag.fit(X_train, y_train)
    
    print("Best parameters set found on development set : ")
    print()
    print(bag.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = bag.cv_results_['mean_test_score']
    stds = bag.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, bag.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, bag.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

Automatically created module for IPython interactive environment
Tuning Hyperparameters for r2

Fitting 7 folds for each of 48 candidates, totalling 336 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  74 tasks      | elapsed:    1.8s
[Parallel(n_jobs=7)]: Done 323 out of 336 | elapsed:    7.7s remaining:    0.2s
[Parallel(n_jobs=7)]: Done 336 out of 336 | elapsed:    8.0s finished


Best parameters set found on development set : 

{'max_features': 10, 'max_samples': 20, 'n_estimators': 100, 'random_state': 8}


Grid scores on development set:

-0.270 (+/-0.576) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 5, 'random_state': None}
-0.474 (+/-0.410) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 5, 'random_state': 4}
-0.021 (+/-0.063) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 5, 'random_state': 8}
-0.036 (+/-0.069) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 10, 'random_state': None}
-0.132 (+/-0.104) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 10, 'random_state': 4}
-0.031 (+/-0.077) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 10, 'random_state': 8}
-0.009 (+/-0.010) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 50, 'random_state': None}
-0.061 (+/-0.130) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 50, 'random_state': 4}
-0.010 (+/-0.024) for {'max_fe

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  74 tasks      | elapsed:    2.1s
[Parallel(n_jobs=7)]: Done 336 out of 336 | elapsed:   11.2s finished


Best parameters set found on development set : 

{'max_features': 10, 'max_samples': 20, 'n_estimators': 100, 'random_state': 8}


Grid scores on development set:

-2.516 (+/-0.745) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 5, 'random_state': None}
-3.036 (+/-0.769) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 5, 'random_state': 4}
-2.516 (+/-0.650) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 5, 'random_state': 8}
-2.651 (+/-0.687) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 10, 'random_state': None}
-2.803 (+/-0.681) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 10, 'random_state': 4}
-2.507 (+/-0.779) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 10, 'random_state': 8}
-2.575 (+/-0.640) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 50, 'random_state': None}
-2.676 (+/-0.730) for {'max_features': 10, 'max_samples': 1, 'n_estimators': 50, 'random_state': 4}
-2.574 (+/-0.626) for {'max_fe

Bootstrap aggregating, also called bagging, is a machine learning ensemble meta-algorithm designed to improve the stability and accuracy of machine learning algorithms used in statistical classification and regression. It also reduces variance and helps to avoid overfitting.

---

## 8. Multi Layer Perceptron Regression

In [35]:
print(__doc__)

mlp1 = MLPRegressor()

# Set the parameters by cross-validation
parameters = {'activation': ['relu', 'tanh', 'logistic', 'identity'], 'alpha': [0.0001, 0.001, 0.01, 0.1], 
                     'learning_rate': ['constant', 'invscaling', 'adaptive'], 'max_iter': [200, 400, 600, 800]}


scores = ['r2', 'max_error']

for score in scores:
    print("Tuning Hyperparameters for %s" % score)
    print()
    
    mlp= GridSearchCV(mlp1,
                      parameters,
                      cv = 7,
                      n_jobs = 7,
                      verbose = True,
                      scoring = score)          #Grid search performed to find the best fit 
    
    mlp.fit(X_train, y_train)
    
    print("Best parameters set found on development set : ")
    print()
    print(mlp.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = mlp.cv_results_['mean_test_score']
    stds = mlp.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, mlp.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test,mlp.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

Automatically created module for IPython interactive environment
Tuning Hyperparameters for r2

Fitting 7 folds for each of 192 candidates, totalling 1344 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:   33.8s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:  2.7min
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:  5.1min
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:  6.9min
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed: 11.4min
[Parallel(n_jobs=7)]: Done 1344 out of 1344 | elapsed: 12.6min finished


Best parameters set found on development set : 

{'activation': 'logistic', 'alpha': 0.1, 'learning_rate': 'constant', 'max_iter': 400}


Grid scores on development set:

-13.113 (+/-15.566) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 200}
-255.450 (+/-1086.979) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 400}
-23.925 (+/-57.564) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 600}
-15.501 (+/-39.391) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 800}
-28.861 (+/-80.210) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'invscaling', 'max_iter': 200}
-23.300 (+/-42.890) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'invscaling', 'max_iter': 400}
-53.386 (+/-143.836) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'invscaling', 'max_iter': 600}
-139.273 (+/-612.466) for {'activation': 'relu', 

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  36 tasks      | elapsed:   32.7s
[Parallel(n_jobs=7)]: Done 186 tasks      | elapsed:  2.8min
[Parallel(n_jobs=7)]: Done 436 tasks      | elapsed:  5.1min
[Parallel(n_jobs=7)]: Done 786 tasks      | elapsed:  6.9min
[Parallel(n_jobs=7)]: Done 1236 tasks      | elapsed: 11.6min
[Parallel(n_jobs=7)]: Done 1344 out of 1344 | elapsed: 12.9min finished


Best parameters set found on development set : 

{'activation': 'logistic', 'alpha': 0.001, 'learning_rate': 'constant', 'max_iter': 600}


Grid scores on development set:

-26.409 (+/-12.691) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 200}
-30.619 (+/-27.217) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 400}
-52.366 (+/-92.038) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 600}
-46.327 (+/-43.711) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'constant', 'max_iter': 800}
-55.245 (+/-93.850) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'invscaling', 'max_iter': 200}
-46.176 (+/-45.370) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'invscaling', 'max_iter': 400}
-61.360 (+/-91.482) for {'activation': 'relu', 'alpha': 0.0001, 'learning_rate': 'invscaling', 'max_iter': 600}
-20.222 (+/-29.452) for {'activation': 'relu', 'alp

Multilayer perceptrons are often applied to supervised learning problems.

They train on a set of input-output pairs and learn to model the correlation (or dependencies) between those inputs and outputs. Training involves adjusting the parameters, or the weights and biases, of the model in order to minimize error.

---

## 9. K Neighbours Regression

In [36]:

print(__doc__)

knnr1 = KNeighborsRegressor()

# Set the parameters by cross-validation
parameters = [{'n_neighbors': [1, 2, 3, 4, 5], 'weights': ['uniform', 'distance'],
                     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'], 'p': [1, 2, 3, 4, 5]}]

scores = ['r2','v_measure_score']

for score in scores:
    print("Tuning Hyperparameters for %s" % score)
    print()
    
    knnr = GridSearchCV(knnr1,
                      parameters,
                      cv = 7,
                      n_jobs = 7,
                      verbose = True,
                      scoring = score)
    
    knnr.fit(X_train, y_train)
    
    print("Best parameters set found on development set : ")
    print()
    print(knnr.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = knnr.cv_results_['mean_test_score']
    stds = knnr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, knnr.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()
    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, knnr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

Automatically created module for IPython interactive environment
Tuning Hyperparameters for r2

Fitting 7 folds for each of 200 candidates, totalling 1400 fits


[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  74 tasks      | elapsed:    0.8s
[Parallel(n_jobs=7)]: Done 674 tasks      | elapsed:    9.0s
[Parallel(n_jobs=7)]: Done 1269 tasks      | elapsed:   54.7s
[Parallel(n_jobs=7)]: Done 1400 out of 1400 | elapsed:  1.4min finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


Best parameters set found on development set : 

{'algorithm': 'auto', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}


Grid scores on development set:

0.418 (+/-0.072) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
0.418 (+/-0.072) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'distance'}
0.389 (+/-0.067) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
0.389 (+/-0.067) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'distance'}
0.377 (+/-0.068) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 3, 'weights': 'uniform'}
0.377 (+/-0.068) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 3, 'weights': 'distance'}
0.379 (+/-0.069) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 4, 'weights': 'uniform'}
0.379 (+/-0.069) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 4, 'weights': 'distance'}
0.379 (+/-0.064) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 5, 'weights': 'uniform'}
0.379 (+/-0.064) for {

[Parallel(n_jobs=7)]: Done  74 tasks      | elapsed:    0.7s
[Parallel(n_jobs=7)]: Done 674 tasks      | elapsed:    9.7s
[Parallel(n_jobs=7)]: Done 1269 tasks      | elapsed:   56.0s


Best parameters set found on development set : 

{'algorithm': 'auto', 'n_neighbors': 3, 'p': 2, 'weights': 'distance'}


Grid scores on development set:

0.828 (+/-0.020) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
0.843 (+/-0.016) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'distance'}
0.828 (+/-0.022) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}
0.844 (+/-0.022) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 2, 'weights': 'distance'}
0.828 (+/-0.020) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 3, 'weights': 'uniform'}
0.844 (+/-0.019) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 3, 'weights': 'distance'}
0.828 (+/-0.021) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 4, 'weights': 'uniform'}
0.843 (+/-0.019) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 4, 'weights': 'distance'}
0.828 (+/-0.021) for {'algorithm': 'auto', 'n_neighbors': 1, 'p': 5, 'weights': 'uniform'}
0.843 (+/-0.019) for {

[Parallel(n_jobs=7)]: Done 1400 out of 1400 | elapsed:  1.4min finished


In pattern recognition, the k-nearest neighbors algorithm (k-NN) is a non-parametric method used for classification and regression. In both cases, the input consists of the k closest training examples in the feature space. In case of regression, the output is the property value for the object. This value is the average of the values of k nearest neighbors.

---

## 10. Linear Regression

In [37]:
print(__doc__)

lr1 = LinearRegression()

# Set the parameters by cross-validation
parameters = {'fit_intercept': ['true', 'false']}

scores = ['r2', 'max_error']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    lr = GridSearchCV(lr1,
                       parameters,
                       cv = 7,
                       n_jobs = 7,
                       verbose = True,
                       scoring = score)

    lr.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(lr.best_params_)
    print()
    print()
    print("Grid scores on development set:")
    print()
    means = lr.cv_results_['mean_test_score']
    stds = lr.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, lr.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Model Loss:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, lr.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    print("Mean Absolute Error is :", mae)
    print("Mean Squared Error is :", mse)
    print()
    print()

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.


Automatically created module for IPython interactive environment
# Tuning hyper-parameters for r2

Fitting 7 folds for each of 2 candidates, totalling 14 fits
Best parameters set found on development set:

{'fit_intercept': 'true'}


Grid scores on development set:

0.860 (+/-0.028) for {'fit_intercept': 'true'}
0.860 (+/-0.028) for {'fit_intercept': 'false'}

Model Loss:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Mean Absolute Error is : 0.19913359647637752
Mean Squared Error is : 0.09483589058946656


# Tuning hyper-parameters for max_error

Fitting 7 folds for each of 2 candidates, totalling 14 fits
Best parameters set found on development set:

{'fit_intercept': 'true'}


Grid scores on development set:

-1.966 (+/-1.436) for {'fit_intercept': 'true'}
-1.966 (+/-1.436) for {'fit_intercept': 'false'}

Model Loss:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

Mean Ab

[Parallel(n_jobs=7)]: Done  14 out of  14 | elapsed:    0.6s finished
[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done  14 out of  14 | elapsed:    0.0s finished


- Linear Regression is a supervised machine learning algorithm where the predicted output is continuous and has a constant slope
- It’s used to predict values within a continuous range, (e.g. sales, price) rather than trying to classify them into categories (e.g. cat, dog)
- There are two main types: 
  Simple regression
Simple linear regression uses traditional slope-intercept form, where m and b are the variables our algorithm will try to “learn” to produce the most accurate predictions. x represents our input data and y represents our prediction.
  Multivariable regression
A more complex, multi-variable linear equation might look like this, where w represents the coefficients, or weights, our model will try to learn.
##### Advantage
When we know the relationship between the independent and dependent variable have a linear relationship, this algorithm is the best to use because it’s the least complex to compared to other algorithms that also try finding the relationship between independent and dependent variable.

---

# Results

MODELS : 
    1. Gradient Boost Regression (Rohan)
    2. XGB Regression (Rohan)
    3. Decision Tree Regressor (Sampras)
    4. AdaBoost Regression (Sampras)
    5. Support Vector Regression (Anirudh)
    6. Random Forest Regression (Anirudh and Nikhita)
    7. Bagging Regression (Padmini)
    8. Multi Layer Perceptron Regression (Padmini)
    9. K Neighbours Regression (Siddhisa)
    10. Linear Regression (Kartik)

### Loss for each model.

Mean Average Error and Mean Squared Error are taken to be the metric values

#### 1. Gradient Boost Regression <br>
R2 => MAE : 0.20716205733987614,<br> 
      MSE : 0.07837190355696298<br>
ME => MAE : 0.17569757454574747,<br> 
      MSE : 0.06575182949196069<br>


#### 2. XGB Regression<br> 
R2 => MAE : 0.12690409875873562,<br> 
      MSE : 0.030317412370844964<br>
ME => MAE : 0.12690409875873562, <br>
      MSE : 0.030317412370844964<br>


#### 3. Decision Tree Regression<br>
R2 => MAE : 0.17636440090541936, <br>
      MSE : 0.06640400397174259<br>
ME => MAE : 0.4055483617381986,  <br>
      MSE : 0.26968052230549555<br>
    
    
#### 4. AdaBoost Regression<br>
R2 => MAE : 0.19625284198317083,<br> 
      MSE : 0.0634113103863094<br>
ME => MAE : 0.25752995771958126, <br>
      MSE : 0.1044488360623666<br>
    
    
#### 5. Support Vector Regression<br>
R2 => MAE : 0.8055055448816022,<br>  
      MSE : 0.8514573918550156<br>
ME => MAE : 0.46486806656837554, <br>
      MSE : 0.386608765702778<br>
     
     
#### 6. Random Forest Regression<br>
R2 => MAE : 0.1438594432870107,<br>  
      MSE : 0.04138448779500928<br>
ME => MAE : 0.1398591723061244,  <br>
      MSE : 0.038978486650283216<br>


#### 7. Bagging Regression<br>
R2 => MAE : 0.25675270401534034,<br> 
      MSE : 0.11634127747369778<br>
ME => MAE : 0.25675270401534034, <br>
      MSE : 0.11634127747369778<br>
    
    
#### 8. Multi Layer Perceptron Regression<br>
R2 => MAE : 0.5029080322486907,<br>  
      MSE : 0.43223269490953425<br>
ME => MAE : 0.4164254490655345,<br>  
      MSE : 0.29193847961022434<br>


#### 9. K Neighbours Regression<br>
R2 => MAE : 0.33744717006994956,<br> 
      MSE : 0.19257206523325418<br>
ME => MAE : 0.357781532026728,<br>   
      MSE : 0.21630062049061596<br>


#### 10. Linear Regression<br>
R2 => MAE : 0.19913359647637752,<br> 
      MSE : 0.09483589058946656<br>
ME => MAE : 0.19913359647637752,<br> 
      MSE : 0.09483589058946656<br>

From the above values we can see that XGB Regression model has the least loss as compared to other models.

Thus, we can use that for our predictions

In [40]:
y_predict = xgb.predict(X_test)
y_predict

array([4.0844727, 1.469626 , 2.0649877, ..., 2.2283826, 1.9788138,
       3.0365233], dtype=float32)

Accordingly, these should be out predicted car prices in lakhs